# Feature Importance
This python file will be to perform Feature Importance using Shapley values to determine the ranking of features in terms of how much they influence the model in effectively predicting the crime codes. The file will start with preprocessing the data similar to previous learning files then performing feature importance with Shapley values


**Authors:** Kevin Lu, Shrusti Jain, Smeet Patel, Taobo Liao


# Imports and Graph Configurations

In [1]:
import numpy as np
import torch
import torch.nn as nn
import pickle
#%matplotlib notebook
import matplotlib.pyplot as plt
import shap

In [2]:
#for some reason, this needs to be in a separate cell
params={
    "font.size":15,
    "lines.linewidth":5,
}
plt.rcParams.update(params)

In [3]:

!gdown 1GOlOSBBJdWWdh8z2oS19aEROm-3oTB4d
!gdown 1_tvxGoEQEMFialDlTOjGmLaU8xz7_Yom
!gdown 1enYGeEeWNFDr-qHQuCNnb4q6yV7O9cpg
!gdown 1yafkDVoUJd6L2scEZeu1Rdg4de8XqfXo
!gdown 1LGqGZvJ5c86IVeC6mS8P1tEyxT_BDsBg

Downloading...
From: https://drive.google.com/uc?id=1GOlOSBBJdWWdh8z2oS19aEROm-3oTB4d
To: c:\Users\jain9\Predicting_LA_Crimes\src\basic_nn_model.pkl

  0%|          | 0.00/658k [00:00<?, ?B/s]
 80%|███████▉  | 524k/658k [00:00<00:00, 4.62MB/s]
100%|██████████| 658k/658k [00:00<00:00, 5.31MB/s]
Downloading...
From: https://drive.google.com/uc?id=1_tvxGoEQEMFialDlTOjGmLaU8xz7_Yom
To: c:\Users\jain9\Predicting_LA_Crimes\src\basic_nn_model_time_sensitive.pkl

  0%|          | 0.00/659k [00:00<?, ?B/s]
100%|██████████| 659k/659k [00:00<00:00, 6.51MB/s]
100%|██████████| 659k/659k [00:00<00:00, 6.51MB/s]
Downloading...
From: https://drive.google.com/uc?id=1enYGeEeWNFDr-qHQuCNnb4q6yV7O9cpg
To: c:\Users\jain9\Predicting_LA_Crimes\src\basic_nn_model_part12.pkl

  0%|          | 0.00/637k [00:00<?, ?B/s]
 82%|████████▏ | 524k/637k [00:00<00:00, 5.10MB/s]
100%|██████████| 637k/637k [00:00<00:00, 5.64MB/s]
Downloading...
From: https://drive.google.com/uc?id=1yafkDVoUJd6L2scEZeu1Rdg4de8XqfXo
To: c:\

In [4]:
# Basic Neural Network with 3 hidden layers, BatchNorm and dropout
class BasicNN(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(BasicNN, self).__init__()
        self.layers = nn.Sequential(
            # Input layer
            nn.Linear(input_size, hidden_sizes[0]),
            nn.BatchNorm1d(hidden_sizes[0]),
            nn.ReLU(),

            # Hidden Layer 1
            nn.Linear(hidden_sizes[0], hidden_sizes[1]),
            nn.BatchNorm1d(hidden_sizes[1]),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate),

            # Hidden Layer 2
            nn.Linear(hidden_sizes[1], hidden_sizes[2]),
            nn.BatchNorm1d(hidden_sizes[2]),
            nn.ReLU(),

            # Hidden Layer 3 (New layer added)
            nn.Linear(hidden_sizes[2], hidden_sizes[3]),
            nn.BatchNorm1d(hidden_sizes[3]),
            nn.ReLU(),

            # Output layer
            nn.Linear(hidden_sizes[3], output_size),
        )

    def forward(self, x):
        return self.layers(x)


In [5]:
# Load dataframe objects from the Deep Learning notebook
# crime_df_train = pd.read_pickle('/content/train.pkl')
# crime_df_debug = pd.read_pickle('/content/debug.pkl')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

input_size = 297
hidden_sizes = [256, 180, 120, 80]
output_size = 69
learning_rate = 1e-3
dropout_rate = 0.2
milestones = [10, 15]

model = BasicNN(input_size=input_size, hidden_sizes=hidden_sizes, output_size=output_size, dropout_rate=dropout_rate)

# Load pre-trained weights into the model
model.load_state_dict(torch.load('basic_nn_model.pkl', map_location=device))
model.to(device)
model.eval()

BasicNN(
  (layers): Sequential(
    (0): Linear(in_features=297, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=180, bias=True)
    (4): BatchNorm1d(180, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Dropout(p=0.2, inplace=False)
    (7): Linear(in_features=180, out_features=120, bias=True)
    (8): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Linear(in_features=120, out_features=80, bias=True)
    (11): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): Linear(in_features=80, out_features=69, bias=True)
  )
)

# Loading Preprocessed Code

Rather than performing preprocessing each time to extract a single batch of data, we save a randomly selected batch as a .npy file and the feature names as a .pkl file, so as to save on both time and memory and allow results to be reproducible.

In [6]:
batch_data = np.load("ft_import_batch.npy")
with open("feature_names.pkl", 'rb') as labelsfile:
    ungrouped_feature_names = pickle.load(labelsfile)

# Feature Importance

In [7]:
def model_predict(X):
    X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(X_tensor)
        return torch.softmax(outputs, dim=1).cpu().numpy()  # Return probabilities

explainer = shap.Explainer(model_predict, batch_data)

# Compute SHAP values for the batch
shap_values = explainer(batch_data)

PermutationExplainer explainer:  17%|█▋        | 709/4096 [01:53<09:10,  6.15it/s]


KeyboardInterrupt: 

In [ ]:
print(shap)

<module 'shap' from '/usr/local/lib/python3.10/dist-packages/shap/__init__.py'>


In [ ]:
average_shap_values = np.mean(shap_values.values, axis=0)  # Shape: (297, 69)
for target_class in range(69):
    class_shap_values = average_shap_values[:, target_class]  # Shape: (297,)

    sorted_indices = np.argsort(-np.abs(class_shap_values))  # Sort by absolute importance
    sorted_shap_values = class_shap_values[sorted_indices]
    sorted_feature_names = np.array(ungrouped_feature_names)[sorted_indices]

    # Plot the averaged SHAP values for this class
    plt.figure(figsize=(10, 6))
    plt.barh(sorted_feature_names[:20], sorted_shap_values[:20])  # Top 20 features
    plt.xlabel("Average SHAP Value")
    plt.ylabel("Feature")
    plt.title(f"Averaged Feature Importance for Class {target_class}")
    plt.gca().invert_yaxis()  # Invert y-axis for better readability
    plt.tight_layout()
    plt.savefig(f"averaged_summary_plot_class_{target_class}.png")
    plt.close()

Actual feature descriptions:
* Feature 1: Date Occured
* Feature 2: Vict Age Was 0 (Unlisted)
* Feature 3: Vict Age
* Feature 4: Rpt Dist No
* Feature 5: Area
* Feature 6: Time Occurred
* Feature 7: Premise Code
* Features 8-13: Status
* Features 14-92: Weapon Used Cd
* Features 93-112: Vict Descent
* Features 113-117: Vict Sex
* Features 118-297: Mocodes

In [ ]:
mean_abs_shap_values = np.mean(np.abs(shap_values.values), axis=(0, 2))  # Shape: (297,)
# Sort features by their mean absolute SHAP value
sorted_indices = np.argsort(-mean_abs_shap_values)
sorted_shap_values = mean_abs_shap_values[sorted_indices]
sorted_feature_names = np.array(ungrouped_feature_names)[sorted_indices]

most_important_feature = sorted_feature_names[0]
most_important_value = sorted_shap_values[0]

print("\nTop 10 Features Across All Classes:")
for feature, importance in zip(sorted_feature_names[:10], sorted_shap_values[:10]):
    print(f"Feature: {feature}, Mean Absolute SHAP Value: {importance:.4f}")



Top 10 Features Across All Classes:
Feature: Premise Code, Mean Absolute SHAP Value: 0.0069
Feature: Mocode_0344, Mean Absolute SHAP Value: 0.0064
Feature: Vict Age, Mean Absolute SHAP Value: 0.0056
Feature: Mocode_0329, Mean Absolute SHAP Value: 0.0033
Feature: Weapon Used Cd_211.0, Mean Absolute SHAP Value: 0.0028
Feature: Date Occ, Mean Absolute SHAP Value: 0.0025
Feature: Mocode_NaN, Mean Absolute SHAP Value: 0.0017
Feature: Mocode_0416, Mean Absolute SHAP Value: 0.0016
Feature: Time Occurred, Mean Absolute SHAP Value: 0.0012
Feature: Mocode_0325, Mean Absolute SHAP Value: 0.0012


From this trial, we can see that most of the important features are Mocodes. This makes sense because mocodes provides a detailed description of the environment of the crime scene before it happens. Such as "Stranger" or "Domestic Violence" etc.

In [ ]:
true_shap_values = np.empty(12)
true_shap_values[:7] = mean_abs_shap_values[:7]
true_shap_values[7] = mean_abs_shap_values[7:12].sum()
true_shap_values[8] = mean_abs_shap_values[13:91].sum()
true_shap_values[9] = mean_abs_shap_values[92:111].sum()
true_shap_values[10] = mean_abs_shap_values[112:116].sum()
true_shap_values[11] = mean_abs_shap_values[117:].sum()
true_feature_names = ["Date Occ", "Vict Age Was 0", "Vict Age", "Rpt Dist No", "Area", "Time Occurred", "Premise Code", "Status", "Weapon Used Cd", "Vict Descent", "Vict Sex", "Mocodes"]
sorted_true_indices = np.argsort(-true_shap_values)
sorted_true_shap_values = true_shap_values[sorted_true_indices]
sorted_true_feature_names = np.array(true_feature_names)[sorted_true_indices]

most_important_true_feature = sorted_true_feature_names[0]
most_important_true_value = sorted_true_shap_values[0]
print("SHAP Values of Each Feature Ranked")
for feature, importance in zip(sorted_true_feature_names, sorted_true_shap_values):
    print(f"Feature: {feature}, Summed Mean Absolute SHAP Value: {importance:.4f}")

SHAP Values of Each Feature Ranked
Feature: Mocodes, Summed Mean Absolute SHAP Value: 0.0328
Feature: Premise Code, Summed Mean Absolute SHAP Value: 0.0069
Feature: Vict Age, Summed Mean Absolute SHAP Value: 0.0056
Feature: Weapon Used Cd, Summed Mean Absolute SHAP Value: 0.0055
Feature: Date Occ, Summed Mean Absolute SHAP Value: 0.0025
Feature: Vict Descent, Summed Mean Absolute SHAP Value: 0.0018
Feature: Vict Sex, Summed Mean Absolute SHAP Value: 0.0013
Feature: Time Occurred, Summed Mean Absolute SHAP Value: 0.0012
Feature: Rpt Dist No, Summed Mean Absolute SHAP Value: 0.0011
Feature: Vict Age Was 0, Summed Mean Absolute SHAP Value: 0.0009
Feature: Status, Summed Mean Absolute SHAP Value: 0.0008
Feature: Area, Summed Mean Absolute SHAP Value: 0.0002


This confirms our expectation that Mocodes would provide the most help in determining the type of crime. This can be further strengthened by the fact that without Mocodes, we were only able to reach around 50% accuracy. Whereas with Mocodes, we could reach 65-70% accuracy in our deep learning model.

# Part 1-2 Feature Importance

In [ ]:
part12_model = BasicNN(input_size=input_size, hidden_sizes=hidden_sizes, output_size=2, dropout_rate=dropout_rate)

# Load pre-trained weights into the model
part12_model.load_state_dict(torch.load('basic_nn_model_part12.pkl', map_location=device))
part12_model.to(device)
part12_model.eval()

<ipython-input-21-c8e30d2c9368>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  part12_model.load_state_dict(torch.load('/content/basic_nn_model_part12.pkl', map_location=d

BasicNN(
  (layers): Sequential(
    (0): Linear(in_features=297, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=180, bias=True)
    (4): BatchNorm1d(180, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Dropout(p=0.2, inplace=False)
    (7): Linear(in_features=180, out_features=120, bias=True)
    (8): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Linear(in_features=120, out_features=80, bias=True)
    (11): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): Linear(in_features=80, out_features=2, bias=True)
  )
)

In [ ]:
def part12_model_predict(X):
    X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
    part12_model.eval()
    with torch.no_grad():
        outputs = part12_model(X_tensor)
        return torch.softmax(outputs, dim=1).cpu().numpy()  # Return probabilities

explainer = shap.Explainer(part12_model_predict, batch_data)

# Compute SHAP values for the batch
shap_values = explainer(batch_data)

PermutationExplainer explainer: 4097it [03:46, 17.29it/s]


In [ ]:
average_shap_values = np.mean(shap_values.values, axis=0)  # Shape: (297, 69)
for target_class in range(2):
    class_shap_values = average_shap_values[:, target_class]  # Shape: (297,)

    sorted_indices = np.argsort(-np.abs(class_shap_values))  # Sort by absolute importance
    sorted_shap_values = class_shap_values[sorted_indices]
    sorted_feature_names = np.array(ungrouped_feature_names)[sorted_indices]

    # Plot the averaged SHAP values for this class
    plt.figure(figsize=(10, 6))
    plt.barh(sorted_feature_names[:20], sorted_shap_values[:20])  # Top 20 features
    plt.xlabel("Average SHAP Value")
    plt.ylabel("Feature")
    plt.title(f"Averaged Feature Importance for Class {target_class}")
    plt.gca().invert_yaxis()  # Invert y-axis for better readability
    plt.tight_layout()
    plt.savefig(f"averaged_summary_plot_class_{target_class}.png")
    plt.close()

In [ ]:
mean_abs_shap_values = np.mean(np.abs(shap_values.values), axis=(0, 2))  # Shape: (297,)
# Sort features by their mean absolute SHAP value
sorted_indices = np.argsort(-mean_abs_shap_values)
sorted_shap_values = mean_abs_shap_values[sorted_indices]
sorted_feature_names = np.array(ungrouped_feature_names)[sorted_indices]

most_important_feature = sorted_feature_names[0]
most_important_value = sorted_shap_values[0]

print("\nTop 10 Features Across All Classes:")
for feature, importance in zip(sorted_feature_names[:10], sorted_shap_values[:10]):
    print(f"Feature: {feature}, Mean Absolute SHAP Value: {importance:.4f}")



Top 10 Features Across All Classes:
Feature: Mocode_0344, Mean Absolute SHAP Value: 0.1458
Feature: Mocode_0329, Mean Absolute SHAP Value: 0.0518
Feature: Vict Age, Mean Absolute SHAP Value: 0.0378
Feature: Weapon Used Cd_211.0, Mean Absolute SHAP Value: 0.0372
Feature: Mocode_NaN, Mean Absolute SHAP Value: 0.0355
Feature: Mocode_0325, Mean Absolute SHAP Value: 0.0250
Feature: Weapon Used Cd_513.0, Mean Absolute SHAP Value: 0.0232
Feature: Premise Code, Mean Absolute SHAP Value: 0.0223
Feature: Weapon Used Cd_516.0, Mean Absolute SHAP Value: 0.0199
Feature: Vict Age Was 0, Mean Absolute SHAP Value: 0.0197


In [ ]:
true_shap_values = np.empty(12)
true_shap_values[:7] = mean_abs_shap_values[:7]
true_shap_values[7] = mean_abs_shap_values[7:12].sum()
true_shap_values[8] = mean_abs_shap_values[13:91].sum()
true_shap_values[9] = mean_abs_shap_values[92:111].sum()
true_shap_values[10] = mean_abs_shap_values[112:116].sum()
true_shap_values[11] = mean_abs_shap_values[117:].sum()
true_feature_names = ["Date Occ", "Vict Age Was 0", "Vict Age", "Rpt Dist No", "Area", "Time Occurred", "Premise Code", "Status", "Weapon Used Cd", "Vict Descent", "Vict Sex", "Mocodes"]
sorted_true_indices = np.argsort(-true_shap_values)
sorted_true_shap_values = true_shap_values[sorted_true_indices]
sorted_true_feature_names = np.array(true_feature_names)[sorted_true_indices]

most_important_true_feature = sorted_true_feature_names[0]
most_important_true_value = sorted_true_shap_values[0]

for feature, importance in zip(sorted_true_feature_names, sorted_true_shap_values):
    print(f"Feature: {feature}, Summed Mean Absolute SHAP Value: {importance:.4f}")

Feature: Mocodes, Summed Mean Absolute SHAP Value: 0.5251
Feature: Weapon Used Cd, Summed Mean Absolute SHAP Value: 0.0700
Feature: Vict Descent, Summed Mean Absolute SHAP Value: 0.0672
Feature: Vict Sex, Summed Mean Absolute SHAP Value: 0.0382
Feature: Vict Age, Summed Mean Absolute SHAP Value: 0.0378
Feature: Premise Code, Summed Mean Absolute SHAP Value: 0.0223
Feature: Vict Age Was 0, Summed Mean Absolute SHAP Value: 0.0197
Feature: Status, Summed Mean Absolute SHAP Value: 0.0140
Feature: Rpt Dist No, Summed Mean Absolute SHAP Value: 0.0118
Feature: Date Occ, Summed Mean Absolute SHAP Value: 0.0114
Feature: Time Occurred, Summed Mean Absolute SHAP Value: 0.0106
Feature: Area, Summed Mean Absolute SHAP Value: 0.0031


# Feature Importance over Time

In [ ]:
time_sensitive_model = BasicNN(input_size=input_size, hidden_sizes=hidden_sizes, output_size=output_size, dropout_rate=dropout_rate)
# Load pre-trained weights into the model
time_sensitive_model.load_state_dict(torch.load('basic_nn_model_time_sensitive.pkl', map_location=device))
time_sensitive_model.to(device)
time_sensitive_model.eval()

<ipython-input-29-3ec26c83526f>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  time_sensitive_model.load_state_dict(torch.load('/content/basic_nn_model_time_sensitive.pkl'

BasicNN(
  (layers): Sequential(
    (0): Linear(in_features=297, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=180, bias=True)
    (4): BatchNorm1d(180, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Dropout(p=0.2, inplace=False)
    (7): Linear(in_features=180, out_features=120, bias=True)
    (8): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Linear(in_features=120, out_features=80, bias=True)
    (11): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): Linear(in_features=80, out_features=69, bias=True)
  )
)

In [ ]:
def time_sensitive_model_predict(X):
    X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
    time_sensitive_model.eval()
    with torch.no_grad():
        outputs = time_sensitive_model(X_tensor)
        return torch.softmax(outputs, dim=1).cpu().numpy()  # Return probabilities

explainer = shap.Explainer(time_sensitive_model_predict, batch_data)

# Compute SHAP values for the batch
shap_values = explainer(batch_data)

PermutationExplainer explainer: 4097it [04:36, 14.24it/s]


In [ ]:
average_shap_values = np.mean(shap_values.values, axis=0)  # Shape: (297, 69)
for target_class in range(69):
    class_shap_values = average_shap_values[:, target_class]  # Shape: (297,)

    sorted_indices = np.argsort(-np.abs(class_shap_values))  # Sort by absolute importance
    sorted_shap_values = class_shap_values[sorted_indices]
    sorted_feature_names = np.array(ungrouped_feature_names)[sorted_indices]

    # Plot the averaged SHAP values for this class
    plt.figure(figsize=(10, 6))
    plt.barh(sorted_feature_names[:20], sorted_shap_values[:20])  # Top 20 features
    plt.xlabel("Average SHAP Value")
    plt.ylabel("Feature")
    plt.title(f"Averaged Feature Importance for Class {target_class}")
    plt.gca().invert_yaxis()  # Invert y-axis for better readability
    plt.tight_layout()
    plt.savefig(f"averaged_summary_plot_class_{target_class}.png")
    plt.close()

In [ ]:
mean_abs_shap_values = np.mean(np.abs(shap_values.values), axis=(0, 2))  # Shape: (297,)
# Sort features by their mean absolute SHAP value
sorted_indices = np.argsort(-mean_abs_shap_values)
sorted_shap_values = mean_abs_shap_values[sorted_indices]
sorted_feature_names = np.array(ungrouped_feature_names)[sorted_indices]

most_important_feature = sorted_feature_names[0]
most_important_value = sorted_shap_values[0]

print("\nTop 10 Features Across All Classes:")
for feature, importance in zip(sorted_feature_names[:10], sorted_shap_values[:10]):
    print(f"Feature: {feature}, Mean Absolute SHAP Value: {importance:.4f}")



Top 10 Features Across All Classes:
Feature: Premise Code, Mean Absolute SHAP Value: 0.0068
Feature: Mocode_0344, Mean Absolute SHAP Value: 0.0065
Feature: Vict Age, Mean Absolute SHAP Value: 0.0052
Feature: Mocode_0329, Mean Absolute SHAP Value: 0.0033
Feature: Weapon Used Cd_211.0, Mean Absolute SHAP Value: 0.0028
Feature: Date Occ, Mean Absolute SHAP Value: 0.0024
Feature: Mocode_0416, Mean Absolute SHAP Value: 0.0017
Feature: Mocode_NaN, Mean Absolute SHAP Value: 0.0016
Feature: Rpt Dist No, Mean Absolute SHAP Value: 0.0014
Feature: Mocode_0325, Mean Absolute SHAP Value: 0.0012


In [ ]:
true_shap_values = np.empty(12)
true_shap_values[:7] = mean_abs_shap_values[:7]
true_shap_values[7] = mean_abs_shap_values[7:12].sum()
true_shap_values[8] = mean_abs_shap_values[13:91].sum()
true_shap_values[9] = mean_abs_shap_values[92:111].sum()
true_shap_values[10] = mean_abs_shap_values[112:116].sum()
true_shap_values[11] = mean_abs_shap_values[117:].sum()
true_feature_names = ["Date Occ", "Vict Age Was 0", "Vict Age", "Rpt Dist No", "Area", "Time Occurred", "Premise Code", "Status", "Weapon Used Cd", "Vict Descent", "Vict Sex", "Mocodes"]
sorted_true_indices = np.argsort(-true_shap_values)
sorted_true_shap_values = true_shap_values[sorted_true_indices]
sorted_true_feature_names = np.array(true_feature_names)[sorted_true_indices]

most_important_true_feature = sorted_true_feature_names[0]
most_important_true_value = sorted_true_shap_values[0]

for feature, importance in zip(sorted_true_feature_names, sorted_true_shap_values):
    print(f"Feature: {feature}, Summed Mean Absolute SHAP Value: {importance:.4f}")

Feature: Mocodes, Summed Mean Absolute SHAP Value: 0.0341
Feature: Premise Code, Summed Mean Absolute SHAP Value: 0.0068
Feature: Weapon Used Cd, Summed Mean Absolute SHAP Value: 0.0054
Feature: Vict Age, Summed Mean Absolute SHAP Value: 0.0052
Feature: Date Occ, Summed Mean Absolute SHAP Value: 0.0024
Feature: Vict Descent, Summed Mean Absolute SHAP Value: 0.0018
Feature: Rpt Dist No, Summed Mean Absolute SHAP Value: 0.0014
Feature: Vict Sex, Summed Mean Absolute SHAP Value: 0.0013
Feature: Time Occurred, Summed Mean Absolute SHAP Value: 0.0010
Feature: Status, Summed Mean Absolute SHAP Value: 0.0008
Feature: Vict Age Was 0, Summed Mean Absolute SHAP Value: 0.0007
Feature: Area, Summed Mean Absolute SHAP Value: 0.0001
